In [4]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [5]:
data = pd.read_csv('/kaggle/input/chatbot-dataset-topical-chat/topical_chat.csv')
data

In [6]:
data[data.conversation_id == 1]

In [7]:
set(data.sentiment.values)

In [9]:
sent_as_code = {' Angry': -10,
 ' Curious to dive deeper': 5,
 ' Disgusted': -10,
 ' Fearful': -7,
 ' Happy': 10,
 ' Neutral': 0,
 ' Sad': -7,
 ' Surprised': 1}

In [10]:
d = {}
for i in set(data.conversation_id.values):
    if not i in d:
        d[i] = []
    counter = 0
    for text, emotion in zip(data[data.conversation_id == i].message.values, data[data.conversation_id == i].sentiment.values):
        d[i].append((text, sent_as_code[emotion], counter))
        counter = 1 - counter

In [11]:
d[1]

In [12]:
emotion_changing_replies = {'query': [], 'reply': [], 'norm': []}

for dialog_id in d:
    for reply_num in range(2, len(d[dialog_id])):
        initial_state = d[dialog_id][reply_num - 2]
        reply = d[dialog_id][reply_num - 1]
        current_state = d[dialog_id][reply_num]
        if current_state[1] - initial_state[1] > 0:
            emotion_changing_replies['query'].append(initial_state[0])
            emotion_changing_replies['reply'].append(reply[0])
            emotion_changing_replies['norm'].append(current_state[1] - initial_state[1])

In [53]:
emotion_spoiling_replies = {'query': [], 'reply': [], 'norm': []}

for dialog_id in d:
    for reply_num in range(2, len(d[dialog_id])):
        initial_state = d[dialog_id][reply_num - 2]
        reply = d[dialog_id][reply_num - 1]
        current_state = d[dialog_id][reply_num]
        if current_state[1] - initial_state[1] < 0:
            emotion_spoiling_replies['query'].append(initial_state[0])
            emotion_spoiling_replies['reply'].append(reply[0])
            emotion_spoiling_replies['norm'].append(current_state[1] - initial_state[1])

In [13]:
happinness_creators = pd.DataFrame(emotion_changing_replies)
happinness_creators

In [54]:
sadness_creators = pd.DataFrame(emotion_spoiling_replies)
sadness_creators

In [40]:
happinness_creators.to_csv('happiness_provokers')

In [22]:
happinness_creators[happinness_creators.norm == 20]

In [26]:
super_answers = ' '.join(happinness_creators[happinness_creators.norm == 20].reply.tolist())

In [29]:
import re

def clean_text(text):
    text = text.lower()
    text = re.sub(r'[^\sа-яА-Яa-zA-Z0-9@\[\]]',' ',text) # Удаляет пунктцацию
    text = re.sub(r'\w*\d+\w*', '', text) # Удаляет цифры
    text = re.sub('\s{2,}', " ", text) # Удаляет ненужные пробелы
    text = text.replace('\n', ' ')
    return text
clean_text(super_answers)[0:100]

In [35]:
from nltk.tokenize import sent_tokenize, word_tokenize

data = clean_text(super_answers)
words = word_tokenize(data)
print(words[0:20])

In [50]:
import nltk
from nltk.corpus import stopwords

nltk.download('stopwords')
stops = set(stopwords.words('english'))
stops

In [39]:
from collections import Counter

cnt = Counter(clean_text(super_answers).split())
cnt.most_common()[0:10]

In [42]:
cnt_clean = Counter(list(filter(lambda el: el not in stops, clean_text(super_answers).split())))
cnt_clean.most_common()[0:10]

In [45]:
top_happy_words = [word for word, _ in cnt_clean.most_common()[5:50]]
top_happy_words[0:10]

In [47]:
from wordcloud import WordCloud
import matplotlib.pyplot as plt
%matplotlib inline

In [48]:
wordcloud = WordCloud().generate(' '.join(top_happy_words))
plt.imshow(wordcloud, interpolation='bilinear')
plt.axis("off")
plt.show()

In [52]:
not_bad_answers = ' '.join(happinness_creators[happinness_creators.norm <= 5].reply.tolist())
cnt_clean1 = Counter(list(filter(lambda el: el not in stops, clean_text(not_bad_answers).split())))
top_words = [word for word, _ in cnt_clean1.most_common()[5:50]]
wordcloud1 = WordCloud().generate(' '.join(top_words))
plt.imshow(wordcloud1, interpolation='bilinear')
plt.axis("off")
plt.show()

In [58]:
sadness_creators[sadness_creators.norm == -20]

In [60]:
bad_answers = ' '.join(sadness_creators[sadness_creators.norm == -20].reply.tolist())
cnt_clean2 = Counter(list(filter(lambda el: el not in stops, clean_text(bad_answers).split())))
top_words2 = [word for word, _ in cnt_clean2.most_common()[5:50]]
wordcloud2 = WordCloud().generate(' '.join(top_words2))
plt.imshow(wordcloud2, interpolation='bilinear')
plt.axis("off")
plt.show()

In [63]:
len(set(top_happy_words))

In [64]:
len(set(top_words2))

In [65]:
len(set(top_happy_words)-set(top_words2))

In [71]:
bad_words = set(top_words2) - set(top_happy_words)
bad_words

In [72]:
good_words = set(top_happy_words) - set(top_words2)
good_words

In [73]:
def show_cloud(words_collection):
    wc = WordCloud().generate(' '.join(words_collection))
    plt.imshow(wc, interpolation='bilinear')
    plt.axis("off")
    plt.show()

In [74]:
show_cloud(bad_words)

In [75]:
show_cloud(good_words)